In [26]:
import torch
from torch import nn
from d2l import torch as d2l

In [27]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [28]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [29]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [30]:
# 边缘检测图初始化
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [31]:
# 垂直边缘检测卷积核
K = torch.tensor([[1.0,-1.0]])

In [32]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [33]:
# .t() 是 .transpose函数的简写版本，只能对2维以下的tensor进行转置
# .T 是 .permute 函数的简化版本，可以对n维tensor进行转置。
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

卷积核学习

In [34]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # [:] 这个可以省略
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 8.265
epoch 4, loss 1.475
epoch 6, loss 0.284
epoch 8, loss 0.063
epoch 10, loss 0.017


In [35]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9959, -0.9749]])

练习

1

In [36]:
# torch.eye 创建对角矩阵，第一个参数是行，第二个参数是列
X = torch.eye(6, 6, dtype=torch.float32)
K = torch.tensor([[1.0, -1.0]])
# (a)
res_a = corr2d(X, K)

# (b)
res_b = corr2d(X.T, K)

# (c)
res_c = corr2d(X, K.T)

res_a, res_b, res_c

(tensor([[ 1.,  0.,  0.,  0.,  0.],
         [-1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  1.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.],
         [ 0.,  0.,  0., -1.,  1.],
         [ 0.,  0.,  0.,  0., -1.]]),
 tensor([[ 1.,  0.,  0.,  0.,  0.],
         [-1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  1.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.],
         [ 0.,  0.,  0., -1.,  1.],
         [ 0.,  0.,  0.,  0., -1.]]),
 tensor([[ 1., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  1., -1.,  0.,  0.,  0.],
         [ 0.,  0.,  1., -1.,  0.,  0.],
         [ 0.,  0.,  0.,  1., -1.,  0.],
         [ 0.,  0.,  0.,  0.,  1., -1.]]))